# Hubbard Model

I'm planning on running VQE and Trotter simulation on a Hubbard Hamiltonian to better understand its structure. 

The Hubbard model is a simplification of the interactions of electrons in a solid. It can be used to explain how these interactions yield insulating, magnetic, and superconducting effects (though I don't understand any of this yet). 

In [1]:
import numpy as np
import scipy.linalg
import scipy.optimize 

from utils import *

In [2]:
from decimal import * 

In [3]:
getcontext().prec=40

In [4]:
getcontext()

Context(prec=40, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

## Defining creation and annihilation operators 

Since we're describing electrons, the Hubbard Hamiltonian's creation and annihilation operators have the normal fermionic anticommutation relations, namely that for a fermion on site $j,k$ with spin $\sigma, \pi$, 

$$ \large \{ c_{j \sigma}, c^\dagger_{k \pi } \} = \delta_{jk} \delta_{\sigma \pi} \qquad \{ c^\dagger_{j\sigma}, c^\dagger_{k \pi} \}= \{ c_{j\sigma} c_{k\pi} \} = 0  $$

I've defined a FermionicOperator class which subclasses CreationOperator and AnnihilationOperator in operators.py. I tried to document it well so `FermionicOperator?` is helpful. 

In [5]:
class Operator: 
    def __init__(self, _type, spin, qubit, dim):
        self.type = _type 
        self.spin = spin
        self.qubit = qubit
        self.dim = dim
        
class CreationOperator(Operator):
    def __init__(self, spin, qubit, dim):
        super(CreationOperator, self).__init__('creation', spin, qubit, dim)

class AnnihilationOperator(Operator):
    def __init__(self, spin, qubit, dim):
        super(AnnihilationOperator, self).__init__('annihilation', spin, qubit, dim)


In [6]:
FermionicOperator?

Object `FermionicOperator` not found.


In [7]:
CreationOperator?

In [8]:
c_1 = CreationOperator('down', 0, 5)
a_1 = AnnihilationOperator('up', 3, 5)

## Jordan-Wigner Transformation

This is all done in the FermionicOperator class. Do `FermionicOperator._gen_JW_mapping??` to get the explanation of how I implemented it. Whenever a `FermionicOperator` is initialized, this function is called and stored in the `FermionicOperator.JW` attribute. 

In [9]:
FermionicOperator._gen_JW_mapping??

Object `FermionicOperator._gen_JW_mapping` not found.


In [10]:
def encode_JW(operator):
    if not issubclass(type(operator), Operator):
        raise TypeError("operator must be an Operator")
        
    result = np.array([[1.0]], dtype='complex128') 
    string_repr = ''
    # Notice the order of the tensor product: gate on 0th qubit is "leftmost" in product
    for i in range(operator.dim): 
        if i < operator.qubit: 
            result = NKron(result, Z)
            string_repr = string_repr + 'Z'
        elif i == operator.qubit and operator.type == 'creation': 
            result = NKron(result, (X -1j*Y) / 2)
            string_repr = string_repr + '-'
        elif i == operator.qubit and operator.type == 'annihilation': 
            result = NKron(result, (X +1j*Y) / 2) 
            string_repr = string_repr + '+'
        elif i > operator.qubit: 
            result = NKron(result, I) 
            string_repr = string_repr + 'I'
        else: 
            raise ValueError("Something is wrong with this code!")
    #print(string_repr)
    return result


## Defining the Hubbard Hamiltonian

Okay, now that we have those preliminaries out of the way, what is the Hubbard Hamiltonian? What's the motivation behind it? How is it useful? 

Answer these. 

The Hubbard Hamiltonian: 
$$ \large H = -t \sum_{<j, k>, \sigma} \Big( c^\dagger_{j\sigma} c_{k\sigma} + c^\dagger_{k \sigma} c_{j \sigma} \Big) + U \sum_j n_{j \uparrow} n_{j \downarrow} - \mu \sum_j \Big(n_{j\uparrow} + n_{j\downarrow} \Big) $$

The first term is kinetic energy, a fermion moving from one site to another. The symbol $<j, k>$ implies iterating over sites that are adjacent. 

The second term is interaction energy, additional energy for a doubly-occupied site. 

The third term is chemical potential, which controls the filling. **Hm, a lot of places don't have this third term (Wikipedia even). Why?**

### 2D Hubbard Hamiltonian on a Square Lattice 

Here is a model of a 2D Hubbard Hamiltontian on a square lattice from [1811.04476](https://arxiv.org/pdf/1811.04476.pdf). We'll represent it by a 2D array: 2D to get the square structure, and in each square `[i][j]`, a dictionary of 4 elements: 'c-up', 'c-down', 'a-up', and 'a-down'. 

![](images/2d_hubbard.png)

In [11]:
# Make sure N <= 10 or computer explodes!!!!!
N_X = 2
N_Y = 2
N = N_X * N_Y

In [12]:
# We use this to make iterating over neighbors easier 
# NOTE: in HubbardHamiltonian2D, we generate the JW transformation instead of the operator class
square_lattice = [[ {'i': i, 'j': j, 
                     'c-up': CreationOperator('up', N_Y * i + j, N), 
                     'c-down': CreationOperator('down', N_Y * i + j, N), 
                     'a-up': AnnihilationOperator('up', N_Y * i + j, N), 
                     'a-down': AnnihilationOperator('down', N_Y * i + j, N)
                    } for j in range(N_X)] for i in range(N_Y)]

In [13]:
np.array(square_lattice).shape

(2, 2)

In [14]:
# TODO: does c-up and c-down act on different qubits? I think they do. But then which qubits are "next" to them 
# in the lattice? 
# I think they need to be acting on different qubits (otherwise they're the same operator...) and even though I 
# make the spin DOWN qubits all after the spin UP qubits, as long as I keep this square lattice 2d array, I 
# should be fine I THINK. Let's try this. So spin DOWN acts on self.N + self.N_Y * i + j and dimension is 
# 2 * self.N
# HOLY COW, I was adding to identity instead of zero!!!!

In [53]:
class HubbardHamiltonian2D: 
    def __init__(self, t, U, mu, N_X, N_Y):
        self.t = t 
        self.U = U 
        self.mu = mu 
        self.N_X = N_X 
        self.N_Y = N_Y
        self.N = N_X * N_Y 
        self.square_lattice = self._gen_square_lattice()
        self.ham_sets = [self._gen_hor_even_sum(), self._gen_hor_odd_sum(), 
                         self._gen_ver_even_sum(), self._gen_ver_odd_sum(), 
                         self._gen_interaction_energy()]
        self.hamiltonian = self._gen_hamiltonian()
        
    def _gen_square_lattice(self):
        # We use this to make iterating over neighbors easier 
        return [[ {'i': i, 'j': j, 
                   # LOOK AT COMMENT ABOVE!
#                    'c-up': encode_JW(CreationOperator('UP', self.N_Y * i + j, self.N)), 
#                    'c-down': encode_JW(CreationOperator('DOWN', self.N_Y * i + j, self.N)), 
#                    'a-up': encode_JW(AnnihilationOperator('UP', self.N_Y * i + j, self.N)), 
#                    'a-down': encode_JW(AnnihilationOperator('DOWN', self.N_Y * i + j, self.N))
                   'c-up': encode_JW(CreationOperator('up', self.N_Y * i + j, 2 * self.N)), 
                   'c-down': encode_JW(CreationOperator('down', self.N + self.N_Y * i + j, 2 * self.N)), 
                   'a-up': encode_JW(AnnihilationOperator('up', self.N_Y * i + j, 2 * self.N)), 
                   'a-down': encode_JW(AnnihilationOperator('down', self.N + self.N_Y * i + j, 2 * self.N))
                  } for j in range(self.N_X)] for i in range(self.N_Y)]
    
    def _gen_hamiltonian(self): 
        return (-self.t * (self.ham_sets[0] + self.ham_sets[1] + 
                           self.ham_sets[2] + self.ham_sets[3]) + 
                self.U * self.ham_sets[4])
    
    # I'm calculating the Hamiltonian like this because we'll be separating these terms later for the ansatz
    def _gen_hor_even_sum(self): 
#         total = np.eye(2**(2 * self.N), dtype='complex128')
        total = np.zeros((2**(2 * self.N), 2**(2*self.N)), dtype='complex128')
        test = 0
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                if j % 2 == 0 and j + 1 < self.N_X: 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i][j+1]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
                    test += 1
        print(test)
        return -self.t * total 
    
    def _gen_hor_odd_sum(self): 
#         total = np.eye(2**(2 * self.N), dtype='complex128')
        total = np.zeros((2**(2 * self.N), 2**(2*self.N)), dtype='complex128')
        test = 0
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                if j % 2 == 1 and j + 1 < self.N_X: 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i][j+1]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
                    test += 1
        print(test)
        return -self.t * total 
    
    def _gen_ver_even_sum(self):
#         total = np.eye(2**(2 * self.N), dtype='complex128')
        total = np.zeros((2**(2 * self.N), 2**(2*self.N)), dtype='complex128')
        test = 0
        for i in range(self.N_Y): 
            if i % 2 == 0 and i + 1 < self.N_Y: 
                for j in range(self.N_X): 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i+1][j]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
                    test += 1
        print(test)
        return -self.t * total 
    
    def _gen_ver_odd_sum(self): 
#         total = np.eye(2**(2 * self.N), dtype='complex128')
        total = np.zeros((2**(2 * self.N), 2**(2*self.N)), dtype='complex128')
        test = 0 
        for i in range(self.N_Y): 
            if i % 2 == 1 and i + 1 < self.N_Y: 
                for j in range(self.N_X): 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i+1][j]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
                    test += 1 
        print(test)
        return -self.t * total 
    
    def _gen_interaction_energy(self):
#         total = np.eye(2**(2 * self.N), dtype='complex128')
        total = np.zeros((2**(2 * self.N), 2**(2*self.N)), dtype='complex128')
        test = 0
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                site = self.square_lattice[i][j] 
                total += NDot(site['c-up'], site['a-up'], site['c-down'], site['a-down'])
                test += 1
        print(test)
        return self.U * total

In [54]:
x = HubbardHamiltonian2D(1, 2, 0, N_X, N_Y)

2
0
2
0
4


In [55]:
print(np.argmin(x.hamiltonian[10]))
print(min(x.hamiltonian[10]))
print(x.hamiltonian.shape)

0
0j
(256, 256)


In [56]:
eig = np.linalg.eigh(x.hamiltonian)
index = np.argmin(eig[0])
print('Ground state energy: ', eig[0][index])
gstate = np.transpose(NKron(eig[1][:,index]))
gstate = gstate / np.linalg.norm(gstate)
print('Ground state: ', gstate)
print(gstate.shape)

Ground state energy:  -3.418550718873847
Ground state:  [[ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 3.36818413e-18+0.j]
 [ 1.40922599e-01+0.j]
 [-2.61360363e-01+0.j]
 [-5.55111512e-17+0.j]
 [-2.61360363e-01+0.j]
 [ 0.00000000e+00+0.j]
 [ 2.22044605e-16+0.j]
 [ 0.00000000e+00+0.j]
 [ 3.05814229e-01+0.j]
 [ 3.72021310e-17+0.j]
 [-9.54899728e-17+0.j]
 [ 6.85840856e-19+0.j]
 [ 8.29321602e-17+0.j]
 [-6.85840856e-19+0.j]
 [ 1.82356827e-18+0.j]
 [ 0.00000000e+00+0.j]
 [-2.76138962e-17+0.j]
 [-2.61360363e-01+0.j]
 [ 1.40922599e-01+0.j]
 [ 2.63318488e-18+0.j]
 [ 3.05814229e-01+0.j]
 [ 6.87210459e-17+0.j]
 [-6.34747459e-17+0.j]
 [ 6.85840856e-19+0.j]
 [-2.61360363e-01+0.j]
 

In [57]:
np.dot(np.transpose(np.conj(gstate)), np.dot(x.hamiltonian, gstate))

array([[-3.41855072+0.j]])

## Variational Hamiltonian Ansatz

The VHA is an ansatz deeply connected to time-evolution of the system. It splits the Hamiltonian into sub-operators and then does time-evolution for those operators: 
$$\large U(\theta) = \prod_{k=1}^n \prod_{\alpha=1}^N \exp \Big( i\theta_{\alpha, k} H_{\alpha} \Big) $$
where $H_\alpha$ are the sub-Hamiltonians and $\theta$ are the parameters being optimized. 

**Question:** Is product from $k=1$ to $n$ meant to indicate multiple iterations of the algorithm? So are there only $N$ parameters, or are there $n \cdot N$? I think it's the former, because $\Big( \exp (i \theta_{1 k} H_1) \cdots \exp (i \theta_{N k} H_N) \Big)$ commutes with itself, so we can combine terms.  

**Question:** Why is this a good ansatz? It seems it only has access to states that are evolutions of the state we start with. Why is that a guarantee that it'll approximate the ground state? 

In [1811.04476](https://arxiv.org/pdf/1811.04476.pdf) they use $N=5$, splitting it as I did above: even and odd horizontal hopping terms, even and odd vertical horizontal terms, and the on-site interaction terms. 

In [58]:
def time_evo(ham, t): 
    # returns e^{i*t*ham}
    # scipy.linalg.expm() uses the Pade approximant; try to understand more of how it works because it seems 
    # pretty useful: https://en.wikipedia.org/wiki/Pad%C3%A9_approximant
    return scipy.linalg.expm(-1j * t * ham)

def prod_time_evo(ham_sets, ts):
    result = 1 
    for pair in zip(ham_sets, ts): 
        result *= time_evo(pair[0], pair[1])
    return result

def test_linalg_expm():
    return scipy.linalg.expm(X)

test_linalg_expm()

array([[1.54308063+0.j, 1.17520119+0.j],
       [1.17520119+0.j, 1.54308063+0.j]])

In [41]:
scipy.linalg?

In [67]:
y = [1, 1]
2 * y

[1, 1, 1, 1]

In [77]:
class VHA: 
    def __init__(self, hubbard, start_state): 
        self.ham_sets = hubbard.ham_sets 
        self.hamiltonian = hubbard.hamiltonian
        self.start_state = start_state
        self.start_params = np.array([5e30 for i in range(5)])
            
    def optimize(self):  
        self.energies = []
        
        def energy(t_params): 
            unitary = prod_time_evo(2 * self.ham_sets, np.real(t_params))
            state = np.dot(unitary, self.start_state)
            curr_energy = np.dot(np.transpose(np.conj(state)), np.dot(self.hamiltonian, state))[0][0]
            #print(curr_energy)
            self.energies.append(curr_energy)
            return curr_energy
                    
        self.sol = scipy.optimize.minimize(energy, 2 * self.start_params, method='Powell', 
                                           bounds=[(0, 3) for i in range(2 * len(self.start_params))])

In [69]:
vha = VHA(x, gstate)

In [70]:
vha.optimize()

KeyboardInterrupt: 

In [62]:
print('Minimum energy: ', vha.sol.fun)
print('Number of iterations: ', vha.sol.nfev)
est_g_state = np.dot(prod_time_evo(vha.ham_sets, vha.sol.x), vha.start_state)
print('Estimated ground state: ', est_g_state)
print(vha.sol.x)

Minimum energy:  (-3.418550718873846-5.169878828456423e-26j)
Number of iterations:  619
Estimated ground state:  [[ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 3.36818413e-18-4.53712633e-44j]
 [ 1.40922599e-01+1.78469690e-10j]
 [-2.61360363e-01+7.04133110e-27j]
 [-5.55111512e-17-7.03014137e-26j]
 [-2.61360363e-01+7.04133110e-27j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 2.22044605e-16-8.97318303e-42j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 3.05814229e-01-8.23896636e-27j]
 [ 3.7202131

### Finding Ground State of Non-interacting Terms

VHA works because finding the ground state of non-interacting terms is efficiently computable on a QC and the VHA can evolve from that ground state to the ground state of the entire Hubbard Hamiltonian. 

We'll start by finding the ground state by using numpy to see if this actually work. 

In [63]:
non_interacting_ham = 0
for i in range(4): 
    non_interacting_ham += x.ham_sets[i]

In [64]:
non_int_ham_noise = non_interacting_ham + np.random.normal(size=(256,256))
eig_non = np.linalg.eig(non_int_ham_noise)
index = np.argmin(eig_non[0])
genergy_non = eig_non[0][index]
print('index', index)
print('Lowest energy: ', genergy_non)
gstate_non = np.transpose(NKron(eig_non[1][:,index]))
print(gstate_non.shape)
print(non_int_ham_noise.shape)

index 1
Lowest energy:  (-16.670114579201226+4.123375222843512j)
(256, 1)
(256, 256)


In [65]:
eig_non[0][index]

(-16.670114579201226+4.123375222843512j)

In [66]:
test_prod = np.dot(non_int_ham_noise, gstate_non)
conj = np.conjugate(gstate_non)
res = 0
for i in range(len(conj)):
    con = np.real(conj[i])
    pro = np.real(test_prod[i])
    if np.abs(con) < 1e-20: 
        continue
    elif np.abs(pro) < 1e-20: 
        continue
    else: 
        res += con * pro
#     print(con)
#     print(pro)
#     print(res)
#     print('---') 
#print('Where does the e-31 come from?!?')
res

array([-8.97280997])

In [78]:
# Choose one of the eigenvectors from lowest eigenvalue
vha = VHA(x, gstate_non)
vha.optimize()

/home/warren/anaconda3/lib/python3.6/site-packages/scipy/sparse/linalg/_onenormest.py:156: RuntimeWarning: invalid value encountered in true_divide
  Y /= np.abs(Y)
/home/warren/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [79]:
vha.sol.fun

(nan+nanj)

In [75]:
est_g_state = np.dot(prod_time_evo(2 * vha.ham_sets, vha.sol.x), vha.start_state)
overlap = np.dot(adjoint(est_g_state), gstate)
overlap[0][0]

(-7.2417873441946315e-22-3.9822028703557743e-22j)

In [76]:
vha.sol.x

array([1.57080033+3.28644404e-10j, 6.17585792+0.00000000e+00j,
       1.57076724-1.65767261e-10j, 6.17585792+0.00000000e+00j,
       1.03003953+1.59421843e-02j])

In [ ]:
for ind in range(0, 80): 
    vha = VHA(x, np.transpose(NKron(eig_non[1][:,ind])))
    vha.optimize()
    print(ind, ' : ', np.real(vha.sol.fun))
    if np.real(vha.sol.fun) < -5: 
        print('Got it! Index is', ind)